### NLP Setup

In [ ]:
import pickle
import sys
import spacy

# Load NLP
#sys.path.append('../')

with open('./nlp/nlp.pickle', 'rb') as f:
    nlp = pickle.load(f)

### Import the Template

In [ ]:
from app.templates.meat_sale.nl_template import nl_template

obs = nl_template['obligations']

print('OBLIGATIONS')
for ob in obs:
    print(f'{ob}: {obs[ob]}\n')

In [ ]:
from app.templates.meat_sale.symboleo.contract_template import get_template

contract_template = get_template()

sym_template = contract_template.to_sym()

sym_template_path = './app/templates/meat_sale/symboleo/symboleo_spec.txt'
with open(sym_template_path, 'w') as f:
    f.write(sym_template)

In [ ]:
from app.src.sentence_summarizer import SentenceSummarizer

summarizer = SentenceSummarizer(nlp)

In [ ]:
val = 'at the buyer\'s warehouse'
doc = nlp(val)
summarizer.summarize(doc)

In [ ]:
for nc in doc.noun_chunks:
    print(nc, len(nc.text), type(nc))

## Other

In [ ]:
from app.classes.spec.symboleo_spec import PAtom
from app.src.graph.graph_builder import GraphBuilder
from app.src.graph.graph_visualizer import GraphVisualizer

graph_builder = GraphBuilder()
graph_visualizer = GraphVisualizer()
digraph = graph_builder.build(PAtom)
gv = graph_visualizer.create_viz(digraph.nodes)
gv.show('nx.html')

In [ ]:
key = 'TERMINATION_CONDITION'
value = 'unless such delay exceeds 10 days'

doc = nlp(value)

summarizer.summarize(value)

## Curr

In [ ]:
from app.classes.contract_update_request import ContractUpdateRequest

key = 'DELIVERY_LOCATION'
value = 'at their warehouse'
doc = nlp(value)


req = ContractUpdateRequest(contract_template, key, value, doc)